# El Niño dataset - Topology of Time Series (v2)

In [24]:
# Import the gtda modules
from gtda.time_series import Resampler, SlidingWindow, takens_embedding_optimal_parameters, \
    TakensEmbedding, PermutationEntropy, SingleTakensEmbedding
from gtda.homology import WeakAlphaPersistence, VietorisRipsPersistence
from gtda.diagrams import Scaler, Filtering, PersistenceEntropy, BettiCurve, PairwiseDistance
from gtda.graphs import KNeighborsGraph, GraphGeodesicDistance

from gtda.pipeline import Pipeline

import numpy as np
from sklearn.metrics import pairwise_distances

import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objects as go
init_notebook_mode(connected=True)

# gtda plotting functions
from gtda.plotting import plot_heatmap
# Plotting functions
from gtda.plotting import plot_point_cloud

from sklearn.decomposition import PCA


## Data 

In [25]:
pc_ENSO = np.loadtxt("/Users/jeannefernandez/Dropbox/Optimal_representatives_weather/data/ENSO/nino3all", skiprows=1)

In [26]:
fig = px.line( title='ENSO temperature estimation, between 500 and 2006')
fig.add_scatter(x = pc_ENSO[:, 0],y=pc_ENSO[:, 1], name='X')
fig.show()

## Fixed parameter analysis

In [31]:
x_fixed = pc_ENSO[:, 0]
y_fixed = pc_ENSO[:, 1]

In [38]:
embedding_dimension_fixed = 3
embedding_time_delay_fixed = 8
stride = 1

embedder_fixed = SingleTakensEmbedding(
    parameters_type="fixed",
    n_jobs=2,
    time_delay=embedding_time_delay_fixed,
    dimension=embedding_dimension_fixed,
    stride=stride,
)

In [39]:
y_fixed_embedded = embedder_fixed.fit_transform(y_fixed)
print(f"Shape of embedded time series: {y_fixed_embedded.shape}")

Shape of embedded time series: (1490, 3)


In [40]:
plot_point_cloud(y_fixed_embedded)

## Non-fixed anylsis

In [42]:
x_nonfixed = pc_ENSO[:, 0]
y_nonfixed = pc_ENSO[:, 1]


In [44]:
max_embedding_dimension_nonfixed = 6
max_embedding_time_delay_nonfixed = 3
stride = 1

embedder_nonfixed = SingleTakensEmbedding(
    parameters_type="search",
    n_jobs=2,
    time_delay=max_embedding_time_delay_nonfixed,
    dimension=max_embedding_dimension_nonfixed,
    stride=stride,
)

y_nonfixed_embedded = embedder_nonfixed.fit_transform(y_nonfixed)

plot_point_cloud(y_nonfixed_embedded)

## Persistence Diagrams

In [45]:
y_fixed_embedded = y_fixed_embedded[None, :, :]
y_nonfixed_embedded = y_nonfixed_embedded[None, :, :]

In [47]:
# 0 - connected components, 1 - loops, 2 - voids
homology_dimensions = [0, 1]# 2]

fixed_persistence = VietorisRipsPersistence(
    homology_dimensions=homology_dimensions, n_jobs=6
)
print("Persistence diagram for fixed parameters")
fixed_persistence.fit_transform_plot(y_fixed_embedded)


Persistence diagram for fixed parameters


array([[[0.        , 0.00611291, 0.        ],
        [0.        , 0.00670971, 0.        ],
        [0.        , 0.00721787, 0.        ],
        ...,
        [0.03058156, 0.03243028, 1.        ],
        [0.02833139, 0.03176297, 1.        ],
        [0.02775988, 0.03094876, 1.        ]]])

In [48]:

nonfixed_persistence = VietorisRipsPersistence(
    homology_dimensions=homology_dimensions, n_jobs=6
)
print("Persistence diagram for nonfixed parameters")
nonfixed_persistence.fit_transform_plot(y_nonfixed_embedded);

Persistence diagram for nonfixed parameters
